In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.set_option("display.max_columns", 40)
pd.options.display.width=None
pd.options.display.float_format = '{:,.2f}'.format

# Explicación de las columnas

- **id:** Id de base de datos del registro
- **name:** Nombre de la publicación
- **operation_type:** Tipo de operación que se esta realizando con la propiedad
- **operation_currency:** Moneda de la operación
- **operation_amount:** Monto de la operación
- **expenses_currency:** Moneda de las expensas
- **expenses_amount:** Monto de las expensas
- **total_mts:** Total de metros de la propiedad
- **covered_mts:** Total de metros cubiertos de la propiedad
- **rooms:** Cantidad de ambientes
- **bedrooms:** Cantidad de dormitorios
- **bathrooms:** Cantidad de baños
- **garages:** Cantidad de garages
- **antique:** Antiguedad de la propiedad
- **building_layout:** Disposición del edificio
- **orientation:** Orientación de la propiedad
- **number_of_floors:** Cantidad de pisos del edificio
- **apartments_per_floor:** Departamentos por cada piso
- **real_estate_type:** Tipo de propiedad
- **posting_type:** Tipo de publicación
- **publisher_id:** Id de base de datos del publicador
- **publisher_name:** Nombre del publicador
- **address:** Dirección de la propiedad
- **city_id:** Id de base de datos del barrio/localidad de la propiedad
- **city:** Nombre del barrio de la propiedad
- **state_id:** Id de base de datos de la provincia de la propiedad
- **state:** Nombre de la provincia de la propiedad
- **country_id:** Id de base de datos del país de la propiedad
- **country:** Nombre del país de la propiedad
- **latitude:** Ubicación en latitud de la propiedad
- **longitude:** Ubicación en longitud de la propiedad
- **reserved:** Tiene reserva?
- **publication_antiquity:** Antiguedad de la publicación
- **url:** Dirección web de la publicación

- TODO: Agregar mas informacion sobre a que corresponde segun el codigo civil el real_estate_type
- TODO: Analizar por que terrenos tienen metros cubiertos
- TODO: En la eliminacion de datos por outlier (percentil 0.999), una idea mas copada seria calcular el percentil de cada variable agrupando antes por cantidad de ambientes por ejemplo.
- TODO: Ver si no conviene a la hora de eliminar registros profundizar el análisis por tipo de propiedad y barrio
- TODO: Calcular la media de las propiedades segun tipo de propiedad, cantidad de ambientes y barrio
- TODO: Ver que hacer con building_layout y orientation ya que hay pocos datos pero pueden ser variables interesantes
- TODO: En la parte de unificación de moneda, podemos armar un script para traer los distintos tipos de cambio y usar un promedio (Lo tengo hecho en otro lado). Ver opción de armar campo Fcha de publicción pra convertir expensa a valor dolar de ese momento
- TODO: Analizar la relación de expensa vs precio de venta
- TARGET: predecir precio de venta en usd
- TODO: ver vble antique, si la tomamos como categorica. Como tratar "en construccion" y "a estrenar". Cuanto impacta en el precio
- TODO: ELimnar aquellos registros que tienen 0 en valor de venta, ya que es nuestro target

# Lectura del Dataset

In [5]:
df = pd.read_excel("/content/Listings.xlsx")
df.sample(5)

,id,name,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,number_of_floors,apartments_per_floor,real_estate_type,posting_type,publisher_id,publisher_name,address,city_id,city,state_id,state,country_id,country,latitude,longitude,reserved,publication_antiquity,url
1869,50579689,Departamento - Venta - 4 Amb - Balcón - Frente...,Venta,USD,"280,000.00",ARS,"127,000.00",130.00,126.00,4.00,3.00,2.00,1.00,50,Frente,N,NaN,NaN,Apartamento,PROPERTY,17062328,Belga Inmobiliaria S.A.,Echeverria al 2100 - Piso 4,V1-C-1003652,Belgrano,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.56,-58.45,False,Publicado hace más de 1 año,https://www.zonaprop.com.ar/propiedades/clasif...
1040,49334442,Quartier Lacroze,En Pozo,USD,"686,166.00",NaN,NaN,164.00,139.00,4.00,3.00,2.00,1.00,NaN,NaN,NaN,NaN,NaN,Vertical,DEVELOPMENT,17032431,Unité Quartier,Av. Federico Lacroze 1935,V1-C-1003652,Belgrano,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.57,-58.44,False,NaN,https://www.zonaprop.com.ar/propiedades/empren...
1635,50304486,Edificio El Olivo - Departamento de 3 Ambiente...,Venta,USD,"320,000.00",ARS,"105,000.00",85.00,80.00,3.00,2.00,2.00,1.00,15,Contrafrente,NaN,NaN,NaN,Apartamento,PROPERTY,17144890,WYPropiedades,Av.crisologo Larralde al 1600,V1-C-1003697,Núñez,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.55,-58.46,False,Publicado hace más de 1 año,https://www.zonaprop.com.ar/propiedades/clasif...
13571,53461594,"Moderno 4 Ambientes y 2 Cocheras - Terraza, Pi...",Venta,USD,"595,000.00",ARS,"202,000.00",223.00,100.00,4.00,3.00,3.00,2.00,3,Contrafrente,N,NaN,NaN,Apartamento,PROPERTY,17061374,Cuño Propiedades,Mason al 4400,V1-C-1003694,Palermo,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.59,-58.43,False,Publicado hace 30 días,https://www.zonaprop.com.ar/propiedades/clasif...
9918,53208905,Venta Departamento 3 Amb. Apto Prof. Apto Airb...,Venta,USD,"120,000.00",ARS,"37,000.00",76.00,75.00,3.00,2.00,1.00,NaN,70,Frente,O,NaN,NaN,Apartamento,PROPERTY,30508856,Gustavo De Simone,Montevideo al 200,V1-C-1003696,Tribunales,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.61,-58.39,False,Publicado hace 71 días,https://www.zonaprop.com.ar/propiedades/clasif...


In [3]:
!pip install ydata_profiling
from ydata_profiling import ProfileReport


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 17.4 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=6415ac6fb7b151fbcbcc4e9bcc70fe7d30e119cba815f124f5353b77e9d580fd
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [4]:
# Usando pandas
profilingreport = ProfileReport(df, title='dptos', minimal=True)
profilingreport


NameError: name 'df' is not defined

# Limpieza de Datos

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16945 entries, 0 to 16944
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     16945 non-null  int64  
 1   name                   16945 non-null  object 
 2   operation_type         16945 non-null  object 
 3   operation_currency     16912 non-null  object 
 4   operation_amount       16912 non-null  float64
 5   expenses_currency      14182 non-null  object 
 6   expenses_amount        14182 non-null  float64
 7   total_mts              16878 non-null  float64
 8   covered_mts            15883 non-null  float64
 9   rooms                  15304 non-null  float64
 10  bedrooms               13421 non-null  float64
 11  bathrooms              15470 non-null  float64
 12  garages                5066 non-null   float64
 13  antique                15461 non-null  object 
 14  building_layout        10121 non-null  object 
 15  or

## Eliminación de columnas

### Eliminación por imposibilidad de estimar

Existen algunas columnas que tienen muy pocos datos y decidimos eliminarlas ya que no hay forma objetiva de imputarles un valor. Entre ellas tenemos:
- number_of_floors
- apartments_per_floor

In [6]:
df.drop(columns=['number_of_floors','apartments_per_floor'], inplace=True)


In [ ]:
df.head(1)

,id,name,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,real_estate_type,posting_type,publisher_id,publisher_name,address,city_id,city,state_id,state,country_id,country,latitude,longitude,reserved,publication_antiquity,url
0,20374238,Excelente 3 Ambientes - Vista Plena a Plaza - ...,Venta,USD,"158,000.00",ARS,"40,000.00",55.00,55.00,3.00,2.00,1.00,NaN,50,Frente,NE,Apartamento,PROPERTY,17049913,Sanchez Naon,Mendoza 2628,V1-C-1003652,Belgrano,V1-B-6,Capital Federal,V1-A-1,Argentina,-34.56,-58.46,False,Publicado hace más de 1 año,https://www.zonaprop.com.ar/propiedades/clasif...


### Eliminacion por referencia identica

Existen algunas columnas que referencian lo mismo ya que cuenta con el id (Interno de la Base de datos) y el nombre. De estas columnas nos quedaremos solo con el nombre. Entre ellas podemos encontrar a:
- publisher_id
- city_id
- state_id
- country_id

In [7]:
df.drop(columns=['publisher_id','city_id','state_id','country_id','publisher_name','name','address'], inplace=True)

In [ ]:
df.head(1)

,id,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,real_estate_type,posting_type,city,state,country,latitude,longitude,reserved,publication_antiquity,url
0,20374238,Venta,USD,"158,000.00",ARS,"40,000.00",55.00,55.00,3.00,2.00,1.00,NaN,50,Frente,NE,Apartamento,PROPERTY,Belgrano,Capital Federal,Argentina,-34.56,-58.46,False,Publicado hace más de 1 año,https://www.zonaprop.com.ar/propiedades/clasif...


### Eliminacion por no aportar al análisis

La columna de url es una columna que no nos aporta valor para los análisis que realizaremos ni tampoco para la predicción

In [8]:
df.drop(columns=['url'], inplace=True)

## Eliminación de valores no analizables

Eliminamos los registros correspondientes a Vertical, Hotel, Edificio, Horizontal y Fondo de Comercio ya que estos tipos de propiedad refieren a la venta de un edificio en su totalidad, varias unidades en simultaneo o negocios que no son el objeto de análisis

In [9]:
#df = df[~df['real_estate_type'].isin(['Vertical','Hotel','Edificio','Horizontal','Fondo de Comercio','Terrenos'])]
df = df[df['real_estate_type'].isin(['Apartamento','PH','Casa'])]


## Estimación de valores

### Reemplazar por cero

Existen valores que podemos reemplazar por 0 ya que la ausencia de los mismos podemos interpretar que refiere a la no existencia de los mismos.
Es una variable que el vendedor public, por lo que asumimos que los valores faltantes son de publicaciones "sin cochera" que no completan ese dato


In [10]:
df['garages'] = df['garages'].fillna(0)

En el caso de bedrooms y rooms, el análisis requiere profundizar aún más ya que según el tipo de propiedad podemos asumir distintas cosas

Si rooms + bedrroms + bathrooms es NAN, elimino el valor por falta de info certera

In [11]:
mascara = df[['rooms','bedrooms','bathrooms']].isna().all(axis=1)

df = df[~mascara]


In [13]:
df

,id,operation_type,operation_currency,operation_amount,expenses_currency,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,building_layout,orientation,real_estate_type,posting_type,city,state,country,latitude,longitude,reserved,publication_antiquity
0,20374238,Venta,USD,"158,000.00",ARS,"40,000.00",55.00,55.00,3.00,2.00,1.00,0.00,50,Frente,NE,Apartamento,PROPERTY,Belgrano,Capital Federal,Argentina,-34.56,-58.46,False,Publicado hace más de 1 año
1,21023929,Venta,USD,"185,000.00",ARS,"87,000.00",84.00,84.00,4.00,3.00,1.00,0.00,50,NaN,NaN,Apartamento,PROPERTY,Recoleta,Capital Federal,Argentina,-34.59,-58.40,False,Publicado hace más de 1 año
2,22003010,Venta,USD,"695,000.00",ARS,"89,000.00",140.00,140.00,4.00,3.00,3.00,1.00,10,Frente,N,Apartamento,PROPERTY,Palermo,Capital Federal,Argentina,-34.57,-58.42,False,Publicado hace más de 1 año
4,24339396,Venta,USD,"120,000.00",ARS,0.00,100.00,70.00,3.00,2.00,1.00,0.00,12,Frente,NaN,PH,PROPERTY,Parque Chacabuco,Capital Federal,Argentina,-34.64,-58.43,False,Publicado hace más de 1 año
6,28015090,Venta,USD,"2,100,000.00",ARS,"200,000.00",211.00,211.00,5.00,3.00,3.00,2.00,5,Frente,N,Apartamento,PROPERTY,Palermo,Capital Federal,Argentina,-34.58,-58.40,False,Publicado hace más de 1 año
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16940,53622858,Venta,USD,"345,000.00",ARS,"145,000.00",192.00,129.00,5.00,4.00,3.00,1.00,20,Contrafrente,NE,Apartamento,PROPERTY,Caballito,Capital Federal,Argentina,-34.62,-58.46,False,Publicado hace 5 días
16941,53622861,Venta,USD,"345,000.00",ARS,"145,000.00",192.00,129.00,5.00,4.00,3.00,1.00,20,NaN,NE,Casa,PROPERTY,Flores,Capital Federal,Argentina,-34.62,-58.46,False,Publicado hace 5 días
16942,53623222,Venta,USD,"275,000.00",ARS,"126,000.00",179.00,90.00,4.00,3.00,2.00,0.00,40,NaN,SE,Apartamento,PROPERTY,Belgrano,Capital Federal,Argentina,-34.57,-58.45,False,Publicado hace 5 días
16943,53623267,Venta,USD,0.00,ARS,0.00,450.00,450.00,4.00,3.00,4.00,1.00,A estrenar,NaN,NE,Casa,PROPERTY,Belgrano,Capital Federal,Argentina,-34.57,-58.44,False,Publicado hace 5 días


real_estate_type
Apartamento    1513
PH               27
Casa             16
Name: count, dtype: int64

Pruebo imputacion por KNN

In [14]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputed_array = imputer.fit_transform(df[['rooms','bedrooms','bathrooms']])
df[['rooms','bedrooms','bathrooms']] = imputed_array

In [15]:
df[df['bedrooms'].isna()].real_estate_type.value_counts()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      12401 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13742 non-null  float64
 7   covered_mts            13333 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

En el caso de covered_mts, al igual que bedrooms y rooms, el análisis requiere profundizar aún más ya que según el tipo de propiedad podemos asumir distintas cosas

In [16]:
df[df['covered_mts'].isna()].real_estate_type.value_counts()

real_estate_type
Apartamento    362
Casa            29
PH              26
Name: count, dtype: int64

Pruebo imputacion de covered mts por la media

In [22]:
media_covered = df['covered_mts'].mean()

df['covered_mts'] = df['covered_mts'].fillna(media_covered)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      12401 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13742 non-null  float64
 7   covered_mts            13750 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

### Reemplazar por máximo posible

Para los registros que tengan un valor mayor de metros cubiertos comparado con metros totales, le seteamos como máximo el de metros totales

In [23]:
condition = df['covered_mts'] > df['total_mts']
df.loc[condition, 'covered_mts'] = df.loc[condition, 'total_mts']

In [25]:
condition = df['total_mts'].isna()
df.loc[condition, 'total_mts'] = df.loc[condition, 'covered_mts']

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      12401 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13750 non-null  float64
 7   covered_mts            13750 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

## Unificación de moneda

Existen registros que estan en moneda ARS ($). Estos los transformaremos a USD utilizando el TC seleccionado para asi unificar los análisis

In [26]:
dolar_bna = 909.5
dolar_blue = 1220
currency_rate = (dolar_bna + dolar_blue) / 2
df['operation_amount'] = np.where(df['operation_currency'] == '$', df['operation_amount'] / currency_rate, df['operation_amount'])
df['operation_currency'] = 'USD'
df['expenses_amount'] = np.where(df['expenses_currency'] == 'ARS', df['expenses_amount'] / currency_rate, df['expenses_amount'])
df['expenses_currency'] = 'USD'

## Transformación de columnas string

La columna antique no solo contiene registros con antiguedad en años, sino que tambien contamos con valores como por ejemplo "A estrenar" o "En construcción". Estos los reemplazaremos por 0 y transformaremos la columna en flotante (Podria ser por entero pero como tiene NaN, pandas no lo soporta)

In [27]:
df.antique.value_counts()

antique
A estrenar         3721
En construcción    1656
40                  948
50                  910
45                  458
                   ... 
134                   1
1930                  1
138                   1
99                    1
129                   1
Name: count, Length: 124, dtype: int64

In [40]:
df['antique'] = df['antique'].replace('A estrenar', 0)
df['antique'] = df['antique'].replace('En construcción', -1)

df['antique'] = df['antique'].astype(float)

In [ ]:
#Prueba completo los Nan de antiguedad con valores muy grandes

## Eliminación de Outliers

In [41]:
df.describe()

,id,operation_amount,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,latitude,longitude
count,"13,750.00","13,750.00","12,401.00","13,750.00","13,750.00","13,750.00","13,750.00","13,750.00","13,750.00","13,521.00","13,746.00","13,746.00"
mean,"52,504,865.50","294,948.84",65.49,116.04,96.38,3.10,2.11,1.69,0.46,22.57,-34.47,-58.33
std,"1,691,064.20","415,037.63",214.51,121.16,90.67,1.53,1.12,0.97,1.10,48.23,12.86,12.85
min,"20,374,238.00",0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00,-1.00,-38.42,-107.75
25%,"52,266,741.75","113,000.00",0.00,52.00,46.00,2.00,1.00,1.00,0.00,0.00,-34.61,-58.47
50%,"53,115,942.50","176,700.00",20.66,81.00,70.00,3.00,2.00,1.00,0.00,10.00,-34.59,-58.44
75%,"53,433,061.75","319,000.00",75.14,139.00,112.00,4.00,3.00,2.00,1.00,40.00,-34.57,-58.41
max,"53,623,344.00","12,000,000.00","17,822.68","5,878.00","3,367.00",32.00,18.00,20.00,72.00,"2,024.00","1,470.00","1,440.00"


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13750 entries, 0 to 16944
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     13750 non-null  int64  
 1   operation_type         13750 non-null  object 
 2   operation_currency     13750 non-null  object 
 3   operation_amount       13750 non-null  float64
 4   expenses_currency      13750 non-null  object 
 5   expenses_amount        12401 non-null  float64
 6   total_mts              13750 non-null  float64
 7   covered_mts            13750 non-null  float64
 8   rooms                  13750 non-null  float64
 9   bedrooms               13750 non-null  float64
 10  bathrooms              13750 non-null  float64
 11  garages                13750 non-null  float64
 12  antique                13521 non-null  object 
 13  building_layout        10115 non-null  object 
 14  orientation            6999 non-null   object 
 15  real_es

Eliminaremos aquellos outliers que distorcionan el dataset. Para ello, aplicaremos como regla general eliminar los registros que esten sobre el percentil 0.999. Esto lo haremos para las columnas:
- operation_amount
- expenses_amount
- total_mts
- covered_mts
- rooms
- bedrooms
- bathrooms
- garages

In [42]:
# Lista de columnas que queremos filtrar
columns_to_filter = ['operation_amount', 'expenses_amount', 'total_mts', 'covered_mts', 'rooms', 'bedrooms', 'bathrooms', 'garages', 'antique']

# Calcular el percentil para cada columna en la lista
percentile = df[columns_to_filter].quantile(0.999)

# Filtrar el DataFrame para eliminar valores por encima del percentil en las columnas seleccionadas
for col in columns_to_filter:
    df = df[df[col] <= percentile[col]]

In [43]:
df.describe()

,id,operation_amount,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,latitude,longitude
count,"12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,252.00","12,251.00","12,251.00"
mean,"52,597,277.25","289,793.86",61.38,111.08,93.10,3.07,2.09,1.65,0.43,21.80,-34.57,-58.45
std,"1,556,842.24","355,162.43",112.60,91.52,74.94,1.43,1.05,0.90,0.71,24.92,0.92,0.86
min,"20,374,238.00",0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00,-1.00,-38.42,-107.75
25%,"52,342,580.25","114,187.50",0.00,53.00,46.00,2.00,1.00,1.00,0.00,0.00,-34.61,-58.46
50%,"53,148,917.00","178,000.00",20.66,80.00,70.00,3.00,2.00,1.00,0.00,10.00,-34.59,-58.44
75%,"53,443,952.25","320,000.00",75.14,135.00,110.00,4.00,3.00,2.00,1.00,41.00,-34.57,-58.41
max,"53,623,344.00","4,500,000.00","1,226.83","1,000.00",859.00,12.00,8.00,7.00,10.00,124.00,23.63,-10.31


In [44]:
# Lista de columnas que queremos filtrar
columns_to_filter = ['operation_amount', 'expenses_amount', 'total_mts', 'covered_mts', 'rooms', 'bedrooms', 'bathrooms', 'garages', 'antique']

# Calcular el percentil para cada columna en la lista
percentile = df[columns_to_filter].quantile(0.001)

# Filtrar el DataFrame para eliminar valores por encima del percentil en las columnas seleccionadas
for col in columns_to_filter:
    df = df[df[col] >= percentile[col]]

In [45]:
df.describe()

,id,operation_amount,expenses_amount,total_mts,covered_mts,rooms,bedrooms,bathrooms,garages,antique,latitude,longitude
count,"12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,233.00","12,232.00","12,232.00"
mean,"52,597,560.75","290,073.60",61.40,111.18,93.22,3.07,2.09,1.65,0.43,21.78,-34.57,-58.45
std,"1,555,539.20","355,344.87",112.67,91.53,74.93,1.43,1.05,0.90,0.71,24.92,0.92,0.86
min,"20,374,238.00",0.00,0.00,21.00,18.00,1.00,1.00,1.00,0.00,-1.00,-38.42,-107.75
25%,"52,342,257.00","114,900.00",0.00,53.00,46.00,2.00,1.00,1.00,0.00,0.00,-34.61,-58.46
50%,"53,148,317.00","178,000.00",20.66,80.00,70.00,3.00,2.00,1.00,0.00,10.00,-34.59,-58.44
75%,"53,443,953.00","320,000.00",75.14,135.00,110.00,4.00,3.00,2.00,1.00,41.00,-34.57,-58.41
max,"53,623,344.00","4,500,000.00","1,226.83","1,000.00",859.00,12.00,8.00,7.00,10.00,124.00,23.63,-10.31


## Eliminación de valores faltantes

Eliminamos los registros que no tienen latitud, longitud y antique ya que estos son muy pocos y la perdida no presenta un problema

In [ ]:
df.dropna(subset=['latitude','longitude','antique'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10706 entries, 0 to 16944
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     10706 non-null  int64  
 1   name                   10706 non-null  object 
 2   operation_type         10706 non-null  object 
 3   operation_currency     10706 non-null  object 
 4   operation_amount       10706 non-null  float64
 5   expenses_currency      10706 non-null  object 
 6   expenses_amount        10706 non-null  float64
 7   total_mts              10706 non-null  float64
 8   covered_mts            10706 non-null  float64
 9   rooms                  10706 non-null  float64
 10  bedrooms               10706 non-null  float64
 11  bathrooms              10706 non-null  float64
 12  garages                10706 non-null  float64
 13  antique                10706 non-null  float64
 14  building_layout        8042 non-null   object 
 15  orienta

# Modelado

## Selección de columnas para el modelo

In [ ]:
df_model = df[['operation_type','expenses_amount','total_mts','covered_mts','rooms','bedrooms','bathrooms','garages','real_estate_type','posting_type','city','antique','operation_amount']]
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10706 entries, 0 to 16944
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   operation_type    10706 non-null  object 
 1   expenses_amount   10706 non-null  float64
 2   total_mts         10706 non-null  float64
 3   covered_mts       10706 non-null  float64
 4   rooms             10706 non-null  float64
 5   bedrooms          10706 non-null  float64
 6   bathrooms         10706 non-null  float64
 7   garages           10706 non-null  float64
 8   real_estate_type  10706 non-null  object 
 9   posting_type      10706 non-null  object 
 10  city              10706 non-null  object 
 11  antique           10706 non-null  float64
 12  operation_amount  10706 non-null  float64
dtypes: float64(9), object(4)
memory usage: 1.1+ MB


## Generación de dummies

In [ ]:
#df_model = pd.get_dummies(df_model, columns=['operation_type'], prefix='optype')
df_model = pd.get_dummies(df_model, columns=['real_estate_type'], prefix='restype')
df_model = pd.get_dummies(df_model, columns=['posting_type'], prefix='postype')
df_model = pd.get_dummies(df_model, columns=['city'], prefix='city')

In [ ]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10706 entries, 0 to 16944
Data columns (total 71 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   operation_type             10706 non-null  object 
 1   expenses_amount            10706 non-null  float64
 2   total_mts                  10706 non-null  float64
 3   covered_mts                10706 non-null  float64
 4   rooms                      10706 non-null  float64
 5   bedrooms                   10706 non-null  float64
 6   bathrooms                  10706 non-null  float64
 7   garages                    10706 non-null  float64
 8   antique                    10706 non-null  float64
 9   operation_amount           10706 non-null  float64
 10  restype_Apartamento        10706 non-null  bool   
 11  restype_Casa               10706 non-null  bool   
 12  restype_PH                 10706 non-null  bool   
 13  postype_PROPERTY           10706 non-null  bool   


## Armado del modelo

### Generacion de X e y

In [ ]:
X = df_model.drop(columns=['operation_amount','operation_type'])
y = df_model['operation_amount']

In [ ]:
df.city.value_counts()

city
Palermo                 1858
Belgrano                1413
Caballito                961
Recoleta                 823
Núñez                    485
Villa Urquiza            477
Villa Crespo             359
Almagro                  354
Flores                   329
Puerto Madero            310
Saavedra                 249
Barrio Norte             237
Colegiales               233
Villa Devoto             214
Balvanera                196
Villa del Parque         177
Retiro                   143
Coghlan                  133
Barracas                 115
San Telmo                102
Boedo                    101
San Cristobal             98
Villa Luro                95
Floresta                  90
Villa Pueyrredón          89
Parque Chacabuco          80
Monserrat                 75
Chacarita                 70
Liniers                   69
La Paternal               69
Monte Castro              67
Congreso                  61
Parque Patricios          51
Villa Ortuzar             51
San Nicol

### Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### Regresión Lineal

In [ ]:
# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

Mean Squared Error: 5.184631170302192e+33
R^2 Score: -3.59317934551618e+22


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_dptos = Ridge()

grid = GridSearchCV(ridge_dptos,
                    {"alpha": np.linspace(0, 20, 1000)},
                    refit=True,
                    cv=5,
                    scoring='neg_mean_absolute_error')
grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': array([ 0.        ,  0.02002002,  0.04004004,  0.06006006,  0.08008008,
        0.1001001 ,  0.12012012,  0.14014014,  0.16016016,  0.18018018,
        0.2002002 ,  0.22022022,  0.24024024,  0.26026026,  0.28028028,
        0.3003003 ,  0.32032032,  0.34034034,  0.36036036,  0.38038038,
        0.4004004 ,  0.42042042,  0.44044044,  0.46046046,  0.48048048,
        0.5005005 ,  0.52052052,  0.5...
       19.41941942, 19.43943944, 19.45945946, 19.47947948, 19.4994995 ,
       19.51951952, 19.53953954, 19.55955956, 19.57957958, 19.5995996 ,
       19.61961962, 19.63963964, 19.65965966, 19.67967968, 19.6996997 ,
       19.71971972, 19.73973974, 19.75975976, 19.77977978, 19.7997998 ,
       19.81981982, 19.83983984, 19.85985986, 19.87987988, 19.8998999 ,
       19.91991992, 19.93993994, 19.95995996, 19.97997998, 20.        ])},
             scoring='neg_mean_absolute_error')

In [ ]:
from sklearn.metrics import mean_absolute_error

ridge_dptos_best = grid.best_estimator_

y_pred = ridge_dptos_best.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(f"MAE de testeo fue: {mae}")
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

MAE de testeo fue: 112301.6203562464
R^2 Score: 0.5692754385431026
